# Requirements:

- pip install catboost
- pip install requests
- pip install numpy
- pip install pandas
- pip install feather-format
- pip install tqdm
- pip install scikit-learn
- pip install scipy


# Импорт модулей

In [1]:
import os
# Reduce CPU load. Need to perform BEFORE import numpy and some other libraries.
os.environ['MKL_NUM_THREADS'] = '2'
os.environ['OMP_NUM_THREADS'] = '2'
os.environ['NUMEXPR_NUM_THREADS'] = '2'

import math
import json
import numpy as np
import pandas as pd
from typing import Optional, List, Tuple, Union
from collections import OrderedDict
import requests
from tqdm import tqdm
import re

from catboost import CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cosine, cityblock, euclidean, braycurtis
from sklearn.metrics import log_loss


# Setup logging
import logging
logging.basicConfig(
    format='%(asctime)s [%(levelname)s] %(name)s %(message)s',
    datefmt='%y-%m-%d %H:%M:%S',
    level=logging.DEBUG,
)
log = logging.getLogger('agro')


RANDOM_SEED = 2021

# Общая идея

В предложенном baseline слова используются как токены, без их семантического анализа.
Я считаю, что это плохая идея.

1. Из-за того, что в русском языке много склонений:

   > "корова", "коровы", "корове", "коровье", ... <br/>

2. А также из-за [Диминутивов](https://ru.wikipedia.org/wiki/%D0%94%D0%B8%D0%BC%D0%B8%D0%BD%D1%83%D1%82%D0%B8%D0%B2)
   и [Аугментативов](https://ru.wikipedia.org/wiki/%D0%90%D1%83%D0%B3%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D1%82%D0%B8%D0%B2),
   которые добавляют немножко оттенков,
   но существенно усложняют машинный анализ текста:

   > "коровушка", "коровёнка", "коровка" <br/>
   > "придоил", "додоил", "передоил" 
   
Вместо этого я решил использовать эмбеддинги слов (`embedding`).
То есть, каждому слову ставится в соответствие точка в N-мерном пространстве.
Обычно N: от 100 до 300.

Анализируя расстояние между эмбедингами слов можно постараться извлечь больше смысла,
чем из кучи уникальных токенов.


# Словарь Embeddings для русского языка

Для работы нам потребуются готовые `embeddings` для русских слов.

Есть некоторые доступные для скачивания словари на
[**RusVectores**](https://rusvectores.org/ru/)

Но размер словарей в них: от 150 до 300 тысяч слов, что довольно мало.
Также, не совсем понятны условия их лицензии.

Есть проект [**"Наташа"**](https://github.com/natasha/navec).
Размер словаря: 500k слов. Лицензия: **MIT**, допускает свободное коммерческое использование.

Существует также другой интересный проект:
[**DeepPavlov**](https://docs.deeppavlov.ai/en/0.0.7/intro/pretrained_vectors.html)
, содержащий около 1.5 млн. слов.
Его лицензия: **Apache 2.0** - позволяет свободное коммерческое использование.

С последним я и буду работать.
Нам потребуется скачать весь словарь, размером 4.14Гб, а затем загрузить его в память.

Я набросал для этого небольшой класс, конкретно под эту задачу:

In [2]:

class GloveModel():
    """
    For a given text returns a list of embeddings
    """
    Pat_Split_Text = re.compile(r"[\w']+|[.,!?;]", flags=re.RegexFlag.MULTILINE)
    Unk_Tag: int = -1
    Num_Tag: int = -1

    def __init__(self, substitutions: Optional[str] = None, log: Optional[logging.Logger] = None):
        if log is None:
            log = logging.getLogger()

        # Load Glove Model. Download and convert from text to .feather format (which is much faster)
        glove_file_feather = 'ft_native_300_ru_wiki_lenta_lower_case.feather'
        if not os.path.exists(glove_file_feather):
            glove_file_vec = glove_file_feather.rsplit(os.extsep, 1)[0] + '.vec'
            if not os.path.exists(glove_file_vec):
                log.info('Downloading glove model for russia language from DeepPavlov...')
                self.download_file(
                    'http://files.deeppavlov.ai/embeddings/ft_native_300_ru_wiki_lenta_lower_case/'
                    'ft_native_300_ru_wiki_lenta_lower_case.vec'
                )
                log.info('Done')
            # Load model from .vec file
            log.info('Loading Glove Model from .vec format...')
            self.glove = self.load_glove_model(glove_file_vec, size=300)
            log.info(f'{len(self.glove)} words loaded!')

            log.info('Saving Glove Model to .feather format...')
            self.glove.reset_index().to_feather(glove_file_feather)
        else:
            log.info('Loading Glove Model from .feather format...')
            self.glove = pd.read_feather(glove_file_feather)
            log.info(f'{len(self.glove)} words loaded!')

        log.info('Sorting glove dataframe by words...')
        self.glove.sort_values('word', axis=0, ignore_index=True, inplace=True)
        log.info('Done')

        self.subs_tab = {}
        if isinstance(substitutions, str):
            for line in substitutions.splitlines():
                words = line.strip().lower().split()
                if len(words) < 2:
                    continue
                self.subs_tab[words[0]] = words[1:]
        log.info(f'Using the substitutions table of {len(self.subs_tab)} records')

        """
        Для неизвестных слов я буду использовать embedding слова 'unk'.
        А для чисел - embedding слова 'num'.
        Я не уверен, что авторы DeepPavlov именно так и планировали.
        Но стандартных '<unk>' или '<num>' я там не обнаружил.
        """
        self.Unk_Tag = int(self.glove.word.searchsorted('unk'))
        self.Num_Tag = int(self.glove.word.searchsorted('num'))
        assert self.glove.word[self.Unk_Tag] == 'unk', 'Failed to find "unk" token in Glove'
        assert self.glove.word[self.Num_Tag] == 'num', 'Failed to find "num" token in Glove'
    
    def __len__(self):
        return len(self.glove)
    
    def __getitem__(self, text: str) -> List[np.ndarray]:
        tags = self.text2tags(text, return_offsets=False)
        embeddings = [self.tag2embedding(tag) for tag in tags]
        return embeddings

    @staticmethod
    def download_file(url: str, block_size=4096, file_name: Optional[str] = None):
        """Downloads file and saves it to local file, displays progress bar"""
        with requests.get(url, stream=True) as response:
            if file_name is None:
                if 'Content-Disposition' in response.headers.keys():
                    file_name = re.findall('filename=(.+)', response.headers['Content-Disposition'])[0]
            if file_name is None:
                file_name = url.split('/')[-1]
            expected_size_in_bytes = int(response.headers.get('content-length', 0))
            received_size_in_bytes = 0
            with tqdm(total=expected_size_in_bytes, unit='iB', unit_scale=True, position=0, leave=True) as pbar:
                with open(file_name, 'wb') as file:
                    for data in response.iter_content(block_size):
                        file.write(data)
                        pbar.update(len(data))
                        received_size_in_bytes += len(data)
            if (expected_size_in_bytes != 0) and (expected_size_in_bytes != received_size_in_bytes):
                raise UserWarning(f'Incomplete download: {received_size_in_bytes} of {expected_size_in_bytes}')
    
    @staticmethod
    def load_glove_model(file_name: str, encoding: str = 'utf-8', size: Optional[int] = None) -> pd.DataFrame:
        """
        Loads glove model from text file into pandas DataFrame
        Returns
        -------
        df : pd.DataFrame
            A dataframe with two columns: 'word' and 'embedding'.
            The order of words is preserved as in the source file. Thus it may be unsorted!
        """
        words, embeddings = [], []
        with tqdm(total=os.path.getsize(file_name), unit='iB', unit_scale=True, position=0, leave=True) as pbar:
            with open(file_name, 'r', encoding=encoding) as f:
                first_line = True
                line = f.readline()
                while line:
                    split_line = line.split()
                    line = f.readline()
                    if first_line:
                        first_line = False
                        if len(split_line) == 2:
                            if size is None:
                                size = int(split_line[1])
                            else:
                                assert size == int(split_line[1]), \
                                    f'Size specified at the first line: {int(split_line[1])} does not match: {size}'
                            continue
                    if size is not None:
                        word = ' '.join(split_line[0:-size])
                        embedding = np.array(split_line[-size:], dtype=np.float32)
                        assert len(embedding) == size, f'{line}'
                    else:
                        word = split_line[0]
                        embedding = np.array(split_line[1:], dtype=np.float32)
                        size = len(embedding)
                    words.append(word)
                    embeddings.append(embedding)
                    pbar.update(f.tell() - pbar.n)
        return pd.DataFrame({'word': words, 'embedding': embeddings})

    def word2tag(self, word: str, use_unk=True, use_num=True) -> int:
        tag = self.glove.word.searchsorted(word)
        if tag == len(self.glove):
            return self.Unk_Tag if use_unk else -1
        if self.glove.word[tag] == word:
            return int(tag)
        if use_num:
            try:
                num = float(word)
                return self.Num_Tag
            except ValueError:
                pass
        return self.Unk_Tag if use_unk else -1

    def tag2embedding(self, tag: int) -> np.ndarray:
        return self.glove.embedding[tag]

    def word2embedding(self, word: str) -> np.ndarray:
        tag = self.word2tag(word)
        return self.glove.embedding[tag]
    
    @staticmethod
    def separate_number_chars(s) -> List[str]:
        """
        Does what its name says.
        Examples
        --------
        'october10' -> ['october', '10']
        '123asdad' -> ['123', 'asdad']
        '-12.3kg' -> ['-12.3', 'kg']
        '1aaa2' -> ['1', 'aaa', '2']
        """
        res = re.split(r'([-+]?\d+\.\d+)|([-+]?\d+)', s.strip())
        res_f = [r.strip() for r in res if r is not None and r.strip() != '']
        return res_f
    
    def text2tags(self, text: str, return_offsets=True) -> Union[List[int], Tuple[List[int], List[int]]]:
        text = text.lower()
        tags = []
        offsets = []
        for m in self.Pat_Split_Text.finditer(text):
            # Get next word and its offset in text
            word = m.group(0)
            offset = m.start(0)
            # Current word can be converted to a list of words due to substitutions: 'Iam' -> ['I', 'am']
            # or numbers and letters separations: '123kg' -> ['123', 'kg']
            if word in self.subs_tab:
                words = self.subs_tab[word]
            else:
                words = self.separate_number_chars(word)
            # Get a list of tags, generated on the source word.
            # Note: they all point to the same offset in the original text.
            for word in words:
                tags.append(self.word2tag(word))
                offsets.append(offset)
        if not return_offsets:
            return tags
        return tags, offsets


# Решение проблемы отсутствующих слов

По условиям конкурса:

> Запрещается Использовать ручную *разметку* *тестовых* данных в качестве решения, в т.ч. любые сервисы разметки.

При этом, не вполне ясно определено, что подразумевается под *разметкой* данных.

В любом случае, речь в запрете идёт о **тестовых** данных.

Поэтому, условия конкурса НЕ помешает мне подготовить словарь для исправления некоторых ошибок,
а также для замены некоторых слов, которые отсутствуют в `embeddings` **DeepPavlov**.

In [3]:
SUBSTITUTIONS = """
цинксодержащие цинк содержащие
проглистогонила дала препарат от глистов
проглистогонил дал препарат от глистов
проглистовать дать препарат от глистов
проглистовали дали препарат от глистов
глистогонить дать препарат от глистов
противогельминтные против глистов
спазган обезболивающий препарат
спазгане обезболивающем препарата
спазганом обезболивающим препаратом
чемерицы рвотный препарат
чемерица рвотный препарат
чемерицей рвотным препаратом

седимин железосодерщащий препарат
левомеколь антисептической мазью
левомиколь антисептическая мазь
левомеколью антисептической мазью
левомиколью антисептической мазью
левомеколем антисептической мазью
левомиколем антисептической мазью

левомецитин антибиотик
левомицитин антибиотик
ливомицитин антибиотик
ливомецитин антибиотик
ливомецетин антибиотик

пребиотик пробиотик
пребеотик пробиотик
прибиотик пробиотик
прибеотик пробиотик
прибиотика пробиотик
пробиотика пробиотик
прибеотика пробиотик
пробеотика пробиотик

отел отёл
отелл отёл
оттел отёл
оттелл отёл
отелу отёлу
отеллу отёлу
оттелу отёлу
оттеллу отёлу
отёле родах
отёлл отёл
оттёл отёл
оттёлл отёл
отёллу отёлу
оттёлу отёлу
оттёллу отёлу
оттела отёла
отелла отёла
оттелла отёла
оттёла отёла
отёлла отёла
оттёлла отёла
отёлом отелом
оттелом отелом
отеллом отелом
оттеллом отелом
оттёлом отелом
отёллом отелом
оттёллом отелом
отелы отёлы
отеллы отёлы
оттелы отёлы
оттеллы отёлы
отелов отёлов
отеллов отёлов
оттелов отёлов
оттеллов отёлов
телилась рожала
отелилась родила
отёлилась родила

бурёнке корове
буренке корове
тёлке корове
тёлочке корове
тёлочка телочка
тёлочку корову
укоровы у коровы
телке корове
телки коровы
бычёк бычек
телятки телята
первотелка корова
первотелки коровы
новотельной коровы
коровушки коровы

доим дою
доишь дою
сдаиваю дою
выдаиваю дою
сдаиваем дою
выдаивем дою
додаиваю дою до конца
доились давали молоко
доется доится
выдаивании доении
сцеживал доил
сцеживала доила
доением отбором молока
сдаивание дойка

отпоил напоил
отпоила напоила
отпоили напоили
выпоить напоить
выпоили напоили
пропоить напоить
пропоили напоили
поите давайте пить
поили давали пить

свищик свищ
свищики свищи

гноящийся гнойный
выдрана вырвана

апитит аппетит
аппитит аппетит
апиттит аппетит
апетит аппетит
апеттит аппетит
опетит аппетит
оппетит аппетит
опеттит аппетит
оппеттит аппетит
опитит аппетит
зарастёт зарастет
пощаще почаще
паздбища пастбища
причинай причиной
пречинай причиной
килограм килограмм
килаграм килограмм
килаграмм килограмм

пузатенькая пузатая

абсцез абсцесс
абсцес абсцесс
абсцезс абсцесс
абсцэз абсцесс
абсцэс абсцесс
абсцэзс абсцесс

перестраховываюсь чересчур переживаю
непроходили не проходили

обкололи поставили укол
колили кололи
вколото поставлено
вкалол вколол
кольнул уколол
истыкали прокололи

накосячил ошибся
ветаптеке ветеринарной аптеке
ветаптеки ветеринарной аптеки
ветаптеку ветеринарную аптеку

житкостью жидкостью
рацеоне рационе
худющие худые
здох сдох
скаждым с каждым
четветый четвертый
ожёг ожег
поднятся подняться

захромала начала хромать

искривился стал кривым

расцарапывает царапает
вычесывает чешется

подшатываются шатаются
пошатываются шатаются

ветиринар ветеринар
ветеринат ветеринар
ветеренаров ветеринаров
ветиренаров ветеринаров
ветеренара ветеринара
ветиренара ветеринара
ветеренару ветеринару
ветиренару ветеринару
ветеренаром ветеринаром
ветиренаром ветеринаром
ветеренары ветеринары
ветиренары ветеринары

расслоилось разделилось на слои
разслоилось разделилось на слои
дегтеобразное похожее на деготь
дегтеобразная похожая на деготь
кремообразное похожее на крем
кремообразная похожая на крем

волосики волосы
залысина лысина
облазит линяет

необнаружил не обнаружил
уменя у меня
делоть делать
дилоть делать
дилать делать

зади сзади
взади сзади
взаде сзади

какба как-бы
какбы как-бы

прошупывается прощупывается
прашупывается прощупывается
пращупывается прощупывается

клещь клещ
клешь клещ
клеш клещ
клещь клещ
клещем клещ
клешем клещ

рвотная рвотный

тужится напрягается
тужиться напрягаться
какает испражняется
срет испражняется
срёт испражняется
дрищет испражняется
запоносил начал поносить
дристать поносить

подсохло высохло
нарывать опухать

оттекла отекла
отекшее опухшее
отёкшее опухшее
припух опух
припухло опухло
припухла опухла
опухшая набухшая
апухшая набухшая

вздувает раздувает
воспаленное поврежденное
вспухшие опухшие
расперло опухло

зашибла ушибла

припухлостей шишек
припухлостями шишками
припухлостям шишкам
припухлостях шишках
припушлостям шишкам

покраснений красноты

жидковат жидкий
жидковатый жидкий
жидковато жидко
жиденький жидкий

животина животное
животины животного
животине животному
животиной животным
животиною животным

температурит имеет повышенную температуру
темпиратурит имеет повышенную температуру
тимпературит имеет повышенную температуру
тимпиратурит имеет повышенную температуру
температурить иметь повышенную температуру
темпиратурить иметь повышенную температуру
тимпиратурить иметь повышенную температуру
тимпературить иметь повышенную температуру

покашливает кашляет
подкашливает кашляет
покашливают кашляют
подкашливают кашляют
откашливаются кашляют

покашливал кашлял
подкашливал кашлял
покашливали кашляли
подкашливали кашляли
откашливались кашляли
"""


## Загрузим Glove Model DeepPavlov

In [4]:

glove = GloveModel(substitutions=SUBSTITUTIONS, log=log)


21-11-17 03:32:20 [INFO] agro Downloading glove model for russia language from DeepPavlov...
21-11-17 03:32:20 [DEBUG] urllib3.connectionpool Starting new HTTP connection (1): files.deeppavlov.ai:80
21-11-17 03:32:20 [DEBUG] urllib3.connectionpool http://files.deeppavlov.ai:80 "GET /embeddings/ft_native_300_ru_wiki_lenta_lower_case/ft_native_300_ru_wiki_lenta_lower_case.vec HTTP/1.1" 301 162
21-11-17 03:32:20 [DEBUG] urllib3.connectionpool Starting new HTTPS connection (1): files.deeppavlov.ai:443
21-11-17 03:32:20 [DEBUG] urllib3.connectionpool https://files.deeppavlov.ai:443 "GET /embeddings/ft_native_300_ru_wiki_lenta_lower_case/ft_native_300_ru_wiki_lenta_lower_case.vec HTTP/1.1" 200 4140857273
100%|████████████████████████████████████████████████████████| 4.14G/4.14G [06:09<00:00, 11.2MiB/s]
21-11-17 03:38:29 [INFO] agro Done
21-11-17 03:38:29 [INFO] agro Loading Glove Model from .vec format...
100%|████████████████████████████████████████████████████████| 4.14G/4.14G [02:35<00:00

# Загрузка данных

Заходим на сайт проекта: [https://contest.ds.agro-code.ru/competition](https://contest.ds.agro-code.ru/competition)
и скачиваем файлы **train.zip** и **test.csv.zip**. После чего нужно разархивировать их и положитьв папку проекта.

Тренировочные данные состоят из полей:
- 'text_id',
- 'text'
- 11 полей с таргетами.

Не стоит забывать, что может быть больше одной болезни для каждого случая.

Тестовые же данные содержат только поля:
- 'text_id'
- 'text'.

In [5]:
log.info('Loading train and test datasets...')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
log.info(f'Loaded train: {len(train)} records, and test: {len(test)} records')

train.head(3)

21-11-17 03:41:46 [INFO] agro Loading train and test datasets...
21-11-17 03:41:46 [INFO] agro Loaded train: 294 records, and test: 99 records


,text_id,text,эймериоз,абсцесс,диспепсия молодняка,остертагиоз,мастит,инфекционный ринотрахеит,отёк вымени,тенденит,сибирская язва,лишай,другое
0,0,"Корова, видимо вставая, раздавила себе сосок. ...",0,0,0,0,1,0,0,0,0,0,0
1,1,Корове 8 лет! Месяц назад промеж четвертей вым...,0,0,0,0,0,0,0,0,1,1,1
2,2,"Молоко течёт само у коровы. Что делать, если у...",0,0,0,0,1,0,1,0,0,0,0


In [6]:
test.head(3)

,text_id,text
0,294,Понос у месячных телят. Подскажите методы и сп...
1,295,"Понос у телят, чем лечить? \nЧем можно вылечит..."
2,296,По какой причине у телёнка отнимаются ноги?\nП...


# Обработка текстовых данных

Преобразуем текст (произвольной длины) в набор фич конечной длины.

Для этого придумаем некоторые ключевые слова или фразы, например: симптомы болезней.
Найдём для них соответствующие эмбединги.
Эмбединги каждой фразы объединим в один эмбединг путём усреднения векторов (можно и суммой, но не суть).

Эти ключевые слова, эти симптомы - будут своебразными "маяками", или, если хотите, точками отсчёта.
Каждое текстовое описание *неопределенной длины* мы заменим *конечным* набором расстояний до этих ключевых маяков.

1. При анализе каждого текста мы разобьём его на **токены** - слова и знаки препинания.
2. Далее для каждого токена найдём **эмбединг**. Для отсутствующих слов - 'unk', для чисел - 'num'.
3. Затем вычислим **расстояние** от этого эмбединга до всех ключевых эмбедингов.
   Евклидово расстояние неинформативно в многомерном пространстве.
   Но расстояний много разных бывает. Мы будет вычислять 4 расстояния: ``(cosine, cityblock, euclidean, braycurtis)``
4. При этом, для всего текста будем запоминать только **наименьшие** расстояния до каждого ключевого слова.
5. Таким образом, из текста неограниченнной длины мы получим лишь **фиксированный набор**
   минимальных расстояний до ключевых слов.


In [51]:

# Symptoms keywords (or phrases)
anchors = [
    'кокцидии', 'абсцесс', 'диспепсия', 'гельминтоз', 'мастит', 'ринотрахеит', 'отёк вымени',
    'воспаление сухожилия', 'острая инфекция', 'лишай',
    'вымя', 'сосок', 'доить', 'температура', 'шишка', 'понос', 'запор', 'кал с кровью',
    'краснота', 'слабость', 'вонь', 'буйный', 'не кушает', 'не даёт молоко', 'пьет мочу',
    'не ходит', 'не встает', 'хромает', 'орёт', 'кашляет', 'чихает', 'глаза слезятся',
    'идет пена', 'пахнет аммиаком', 'после отёла', 'вялость', 'аборт', 'свищ', 'гной из раны',
    'кровавая моча', 'выделения из носа', 'рвота', 'истощение', 'судороги', 'расширенные зрачки'
]
anchor_embeddings = [np.mean(np.stack(glove[target]), axis=0) for target in anchors]

distance_functions = (cosine, cityblock, euclidean, braycurtis)

def embedings2features(text_embeddings: List[np.ndarray]) -> pd.Series:
    result = OrderedDict()
    for embedding in text_embeddings:
        for anchor_embedding, anchor in zip(anchor_embeddings, anchors):
            anchor = '_'.join(anchor.split())
            for dist_func in distance_functions:
                feature_name = f'{anchor}_{dist_func.__name__}'
                dist = float(dist_func(embedding, anchor_embedding))
                if feature_name not in result:
                    result[feature_name] = dist
                else:
                    result[feature_name] = min(dist, result[feature_name])
    return pd.Series(result)

def embedings2distances(text_embeddings: List[np.ndarray]) -> Tuple[pd.Series, np.ndarray]:
    result = OrderedDict()
    distances = []
    for embedding in text_embeddings:
        D = np.ones((len(anchor_embeddings), len(distance_functions)), dtype=np.float32) * np.inf
        for i, (anchor_embedding, anchor) in enumerate(zip(anchor_embeddings, anchors)):
            anchor = '_'.join(anchor.split())
            for j, dist_func in enumerate(distance_functions):
                feature_name = f'{anchor}_{dist_func.__name__}'
                dist = float(dist_func(embedding, anchor_embedding))
                D[i, j] = dist
                if feature_name not in result:
                    result[feature_name] = dist
                else:
                    result[feature_name] = min(dist, result[feature_name])
        distances.append(D)
    return pd.Series(result), np.stack(distances, axis=0)

def text2features(text) -> pd.Series:
    text_embeddings = glove[text]
    return embedings2features(text_embeddings)


In [52]:

def init_dataset(ds: pd.DataFrame, file_name: str) -> pd.DataFrame:
    # Check if file already exists
    if not os.path.exists(file_name):
        log.info(f'Constructing new synthetic dataset...')
        tqdm.pandas(position=0, leave=True)
        X: pd.DataFrame = ds['text'].progress_apply(text2features)
        
        data = [ds[['text_id']], X]
        if len(ds.columns) > 2:
            data.append(ds[ds.columns[2:]])
        
        data = pd.concat(data, axis=1, ignore_index=False, copy=False)

        data.to_feather(file_name)
        log.info(f'Saved new dataset to {file_name}')
    
    else:
        data = pd.read_feather(file_name)
        log.info(f'Loaded synthetic dataset from {file_name}')

    assert len(data) == len(ds)
    return data


In [54]:

log.info(f'Constructing new features based on text embeddings...')
data = init_dataset(train, 'train.synth.feather')
log.info(f'New synthetic {len(data.columns)} features:\n{data.columns}')


21-11-17 13:33:48 [INFO] agro Constructing new features based on text embeddings...
21-11-17 13:33:48 [INFO] agro Constructing new synthetic dataset...
100%|████████████████████████████████████████████████████████████| 294/294 [01:08<00:00,  4.29it/s]
21-11-17 13:34:56 [INFO] agro Saved new dataset to train.synth.feather
21-11-17 13:34:56 [INFO] agro New synthetic 192 features:
Index(['text_id', 'кокцидии_cosine', 'кокцидии_cityblock',
       'кокцидии_euclidean', 'кокцидии_braycurtis', 'абсцесс_cosine',
       'абсцесс_cityblock', 'абсцесс_euclidean', 'абсцесс_braycurtis',
       'диспепсия_cosine',
       ...
       'абсцесс', 'диспепсия молодняка', 'остертагиоз', 'мастит',
       'инфекционный ринотрахеит', 'отёк вымени', 'тенденит', 'сибирская язва',
       'лишай', 'другое'],
      dtype='object', length=192)


# Базовая модель

В качестве базового решения используется CatBoostClassifier.

Помимо этого задача является мультилейбл классификацией, поэтому модель обернута в OneVsRestClassifier.

**Делим данные на тренировочную и валидационную выборку**

> Примечание: для финального обучения модели используется вся база.
> Поэтому добавлена строка <br/>
> ``X_train, y_train = data[data.columns[1:-11]], data[data.columns[-11:]]``

In [55]:
data[data.columns[1:-11]]

,кокцидии_cosine,кокцидии_cityblock,кокцидии_euclidean,кокцидии_braycurtis,абсцесс_cosine,абсцесс_cityblock,абсцесс_euclidean,абсцесс_braycurtis,диспепсия_cosine,диспепсия_cityblock,...,истощение_euclidean,истощение_braycurtis,судороги_cosine,судороги_cityblock,судороги_euclidean,судороги_braycurtis,расширением_зрачков_cosine,расширением_зрачков_cityblock,расширением_зрачков_euclidean,расширением_зрачков_braycurtis
0,0.591457,75.867775,5.369314,0.668973,0.452529,77.162872,5.585297,0.554731,0.468694,75.606476,...,4.427354,0.701167,0.519069,73.452789,5.415156,0.600149,0.534728,48.670525,3.587400,0.634327
1,0.469231,75.295456,5.369314,0.540832,0.287436,65.141655,4.740943,0.414336,0.436011,80.544640,...,4.333127,0.678275,0.415106,72.940926,5.250240,0.509341,0.539893,48.670525,3.587400,0.646094
2,0.566464,72.263329,5.283903,0.640890,0.452529,77.280655,5.585297,0.554731,0.468694,75.606476,...,4.333127,0.669660,0.519069,73.452789,5.347677,0.581161,0.534728,48.670525,3.587400,0.634327
3,0.594875,74.417076,5.369314,0.652104,0.601096,83.025307,6.139108,0.653957,0.573224,81.517899,...,4.333127,0.685853,0.513013,68.385338,5.089166,0.588483,0.597966,49.423149,3.666537,0.646094
4,0.600721,73.337669,5.234788,0.679806,0.488348,77.162872,5.655127,0.586208,0.555616,82.095352,...,4.427354,0.627128,0.493355,70.846535,5.115848,0.570865,0.584978,48.670525,3.587400,0.646043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,0.566313,72.735962,5.222890,0.642979,0.415779,73.584251,5.398958,0.524001,0.397364,68.712341,...,4.427354,0.618625,0.418053,65.988342,4.746195,0.528645,0.520999,48.670525,3.587400,0.633650
290,0.652231,75.937576,5.369314,0.683912,0.473130,79.853485,5.912630,0.551172,0.327020,67.312851,...,4.546192,0.640367,0.369335,69.156448,4.860199,0.483241,0.567697,49.423149,3.666537,0.645175
291,0.387000,65.782211,4.707370,0.499236,0.331614,66.181374,4.879470,0.452909,0.393186,69.612961,...,4.427354,0.640238,0.373915,64.838638,4.634337,0.471916,0.498797,48.670525,3.587400,0.607319
292,0.606155,73.337669,5.234788,0.655418,0.312692,67.253563,4.945843,0.429833,0.377769,75.289627,...,4.546192,0.637970,0.285759,62.100117,4.445947,0.406590,0.474234,50.563011,3.666537,0.590178


In [56]:
X_train, X_valid, y_train, y_valid = train_test_split(
    data[data.columns[1:-11]],
    data[data.columns[-11:]],
    test_size=0.2,
    random_state=RANDOM_SEED
)
X_train, y_train = data[data.columns[1:-11]], data[data.columns[-11:]]
log.info(f'Split dataset into train: {len(X_train)} and valid: {len(X_valid)}')

21-11-17 13:35:07 [INFO] agro Split dataset into train: 294 and valid: 59


# Обучаем модель

In [57]:
# Initialize and train new model

estimator = CatBoostClassifier(
    max_depth=12,
    iterations=1000,
    # bootstrap_type=,  # Bayesian, Bernoulli, MVS,
    # boosting_type='Ordered',
    # loss_function='MultiLogloss',
    # eval_metric='MultiLogloss',
    verbose=False,
    thread_count=2,
    allow_writing_files=False,
    random_seed=RANDOM_SEED,
)

model = OneVsRestClassifier(estimator=estimator)
log.info(f'Initialized model: {model}')

log.info(f'Starting training...')
model.fit(X_train, y_train)
log.info('Done')


21-11-17 13:38:21 [INFO] agro Initialized model: OneVsRestClassifier(estimator=<catboost.core.CatBoostClassifier object at 0x0000016F0ECF1DC0>)
21-11-17 13:38:21 [INFO] agro Starting training...
21-11-17 16:35:37 [INFO] agro Done


# Считаем метрику

В качестве ground truth функция принимает на вход датафрейм/массив из 10 столбцов (все классы, кроме "другое").

Предсказанные значения – "prediction", также должны быть либо в виде датафрейма, либо в виде массива.

In [58]:
def log_loss_score(prediction, ground_truth):
    log_loss_ = 0
    ground_truth = np.array(ground_truth)
    for i in range(10):
        log_loss_ += log_loss(ground_truth[:, i], prediction[:, i])
    return log_loss_ / 10


In [59]:
# Get model score on validation set

y_pred = model.predict_proba(X_valid)
score = log_loss_score(ground_truth=y_valid, prediction=y_pred)
log.info(f'Model score on validation set: {score}')


21-11-17 16:35:59 [INFO] agro Model score on validation set: 0.03313410417962717


# Создание файла отправки

1. В файле с ответами должны быть вероятности для каждого класса.
   Сумма вероятностей в каждой строке может быть больше 1.

2. Кроме того, в файле с ответами нужно привести ссылки на символы текста,
   из-за которых модель приняла то или иное решение.

Со вторым требованием есть очевидные сложности.
Ведь в моём решении я работаю не с токенами из текста,
а с некими обобщёнными расстояниями в многомерном пространстве `word embeddings`.

Придётся сначала вычислить важность фич для каждой предсказанной болезни.

Затем вычислить важность фич в конкретном тексте.

Из чего найти слова с ближайшим расстоянием, согласно этим фичам.

In [60]:
# Compute importance of features for different targets

target_features_importance = [estimator.feature_importances_ for estimator in model.estimators_]
target_features_importance = abs(np.stack(target_features_importance, axis=0))
target_features_importance = target_features_importance / target_features_importance.sum(0, keepdims=True)
target_features_importance = np.nan_to_num(target_features_importance, nan=0.0, posinf=1.0, neginf=0.0)
# target_features_importance has shape: (num_targets, num_features)
# for each estimator, in each row, the sum of coefficients is equal to 1.0


In [61]:
# Construct result
log.info('Constructing result...')
result = {}
target_threshold = 0.4
token_threshold = 0.83

with tqdm(total=len(test), position=0, leave=True) as pbar:
    for row in test.itertuples():
        tags, offsets = glove.text2tags(row.text, return_offsets=True)
        text_embeddings = [glove.tag2embedding(tag) for tag in tags]
        input, distances = embedings2distances(text_embeddings)
        # distances has shape: (num_tokens, num_symptoms, num_distance_functions)

        distances = abs(distances.reshape((distances.shape[0], -1)))
        # distances has shape: (num_tokens, num_features)

        output = model.predict_proba([input])[0]
        # output has shape: (num_targets,)
        
        # Zero out some targets below threshold
        targets_importance = output.copy()
        targets_importance[targets_importance < target_threshold] = 0.0
        # targets_importance has shape: (num_targets,)

        features_importance = np.expand_dims(targets_importance, 1) * target_features_importance
        # features_importance has shape: (num_targets, num_features)

        # Invert distances, as we are interested in the closest distance.
        # Note: here I assume that the shorter the distance - the more important the token is.
        # But, in fact, we don't know at what distance the feature becomes important!
        inverted_distances = distances.max(0, keepdims=True) - distances
        inverted_distances = (inverted_distances - inverted_distances.min(0, keepdims=True)) / inverted_distances.max(0, keepdims=True)
        # inverted_distances has shape: (num_tokens, num_features)

        token_importance = inverted_distances @ features_importance.transpose()
        # token_importance has shape: (num_tokens, num_targets)

        token_importance = token_importance.sum(1)
        token_importance = token_importance / token_importance.max()
        token_importance = np.nan_to_num(token_importance, nan=0.0, posinf=1.0, neginf=0.0)
        # token_importance has shape: (num_tokens,) with values in range [0...1]

        span = []
        st, en = None, None
        for i, (importance, offset) in enumerate(zip(token_importance, offsets)):
            # Some consequent tokens address the same starting offset in the text.
            # This is due to words substitutions by phrases for missing words in Glove.
            if (en is not None) and (en == offset):
                continue
            # Current word is not important
            if importance < token_threshold:
                if st is not None:
                    en = offset - 1
                    if en - st > 1:
                        span.append([st, en])
                    st, en = None, None
                continue
            # Current word is the first important word in the span. Keep st
            if st is None:
                st = offset
            # Update en
            en = offset
        if st is not None:
            en = len(row.text) - 1
            if en - st > 1:
                span.append([st, len(row.text) - 1])

        result[str(row.text_id)] = {
            'span': span,
            'label': output[:10].tolist(),
        }
        pbar.update(1)

log.info('Done')


21-11-17 16:36:12 [INFO] agro Constructing result...
 42%|██████████████████████████▎                                   | 42/99 [00:12<00:10,  5.30it/s]Z:\APPDAT~1/ipykernel_13996/1591399476.py:39: RuntimeWarning: invalid value encountered in true_divide
  token_importance = token_importance / token_importance.max()
100%|██████████████████████████████████████████████████████████████| 99/99 [00:26<00:00,  3.79it/s]
21-11-17 16:36:38 [INFO] agro Done


In [62]:
submission_file_name = 'submission.json'

log.info(f'Exporting result to {submission_file_name}')

with open(submission_file_name, 'w') as f:
    json.dump(result, f, indent=4)


21-11-17 16:36:41 [INFO] agro Exporting result to submission.json


In [50]:
glove.word2tag('истощение', False, False)

656841

In [63]:
target_features_importance.shape

(11, 180)